<a href="https://colab.research.google.com/github/delibae/health_care_pr/blob/pr2%2Fbhj/medicine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Proprecessing

In [ ]:
import pandas as pd

## Create DataLoader

## define get_img(cv2)

In [ ]:
import cv2

def get_img_list(series) :

    
    reshaped_image_list = []

    for file_name in series :
        image_path = file_name
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)

        image = cv2.resize(image, dsize=(100, 100), interpolation=cv2.INTER_AREA)

        reshaped_image = image.reshape(3,100,100) #reshape
        reshaped_image_list.append(reshaped_image)
        
    return reshaped_image_list

# define get_transform(normalize)

In [ ]:
def get_transform():

    t = list()
    t.append(transforms.ToTensor())

    t.append(transforms.Normalize(mean=(.5),std=(.5)))
    
    return transforms.Compose(t)

## define dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_list, label=None):
        self.transform = get_transform()
        self.img_list = img_list
        self.label = label
        self.transformed_img_list = list(map(self.transform, self.img_list))

    def __len__(self):
        return len(self.img_list)
    
    def __getitem__(self, i):
        
        x_data = self.transformed_img_list[i]

        return x_data, torch.tensor(self.label[i],dtype = torch.long)

## for local save image load

In [ ]:
import pickle
import gzip

# to define
file_path_l = []

data = get_img_list(file_path_l)

with gzip.open('ImageFile.pickle', 'wb') as f:
    pickle.dump(data, f)

## define dataloader

In [ ]:
import math

def get_dataloader(x, y):
    img_list = get_img_list(x)
    dataset = CustomDataset(img_list=img_list, label=y)
    dataloader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size)
    total_batch = math.ceil(len(dataset)/batch_size)

    return dataloader, total_batch

## create dataloader

In [ ]:
batch_size = 64

x_train = train['file_name']
x_valid = valid['file_name']
y_train = train['label']
y_valid = valid['label']

train_dataloader, train_total_batch = get_dataloader(x_train, y_train)
valid_dataloader, valid_total_batch = get_dataloader(x_valid, y_valid)




# Model

In [ ]:
import torch
import torchvision
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from sklearn.model_selection import StratifiedKFold
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## model structure

In [ ]:
# class CNN(nn.Module):
#     def __init__(self):
#         super(CNN, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=20, kernel_size=5, stride=1)
#         self.conv2 = nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5, stride=1)
#         self.fc1 = nn.Linear(22 * 22 * 50, 500)
#         self.fc2 = nn.Linear(500, 6)

#     def forward(self, x):
#         x = F.relu(self.conv1(x))
#         x = F.max_pool2d(x, kernel_size=2, stride=2)
#         x = F.relu(self.conv2(x))
#         x = F.max_pool2d(x, kernel_size=2, stride=2)
#         x = x.view(-1, 50*22*22)
#         x = F.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x

In [ ]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.conv = nn.Sequential(

            nn.Conv2d(3, 64, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(64, 64, 3, padding=1),nn.LeakyReLU(0.2),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(128, 128, 3, padding=1),nn.LeakyReLU(0.2),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(128, 256, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(256, 256, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(256, 256, 3, padding=1),nn.LeakyReLU(0.2),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(256, 512, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
            nn.MaxPool2d(2, 2)
        )
        #512 4 4
        self.avg_pool = nn.AvgPool2d(4)
        #512 1 1
        self.classifier = nn.Linear(512, 1000)
        """
        self.fc1 = nn.Linear(512*2*2,4096)
        self.fc2 = nn.Linear(4096,4096)
        self.fc3 = nn.Linear(4096,10)
        """

    def forward(self, x):
        print(x.shape)
        features = self.conv(x)
        print(x.shape)
        x = self.avg_pool(features)
        print(x.shape)
        #print(avg_pool.size())
        x = x.view(features.size(0), -1)
        print(x.shape)
        #print(flatten.size())
        x = self.classifier(x)
        print(x.shape)
        #x = self.softmax(x)
        return x, features



## device

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## create model(loss function / optimizer)

In [ ]:
model = VGG().to(device)

model.train()
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)

## test model

In [ ]:
inp = torch.randn(2,3,128,128)
model(inp)

# Train

## start train

In [ ]:
loss_l = []
validloss_l = []
epoch_num = 30

for epoch in range(epoch_num):
  for index, (data, target) in enumerate(train_dataloader):
    data = data.to(device)
    target = target.to(device)
    optimizer.zero_grad()  # 기울기 초기화
    output = model(data)
    loss = criterion(output, target)
    loss.backward()  # 역전파
    optimizer.step()

  for   index, (data, target) in enumerate(valid_dataloader):
    data = data.to(device)
    target = target.to(device)
    output = model(data)
    valid_loss = criterion(output,target)

  print(f"loss of {epoch} epoch, {loss.item():.3f} / valid loss: {valid_loss:.3f}")
  loss_l.append(loss.item())
  validloss_l.append(valid_loss.item())


## plot loss

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(loss_l, label = "train loss")
plt.plot(validloss_l, label = "valid loss")

# Model Analysis

## Confusion Matrix

In [ ]:
model.eval()

In [ ]:
y_pred = []
y_true = []

# iterate over test data
for index, (inputs, labels) in enumerate(valid_dataloader):
        output = model(inputs) # Feed Network

        output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
        y_pred.extend(output) # Save Prediction
        
        labels = labels.data.cpu().numpy()
        y_true.extend(labels) # Save Truth

In [ ]:
# to define
label_list = []

In [ ]:
classes = label_list

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
cf_matrix = confusion_matrix(y_true, y_pred, normalize = 'pred')
df_cm = pd.DataFrame(cf_matrix/np.sum(cf_matrix) *10, index = [i for i in classes],
                     columns = [i for i in classes])
plt.figure(figsize = (12,7))
sns.heatmap(df_cm, annot=True)
plt.show